In [2]:
!pip install selenium

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.5/10.5 MB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 467.2/467.2 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 2.8 MB/s eta 0:00:00


# IMPORTAMOS LIBRERÍAS

In [86]:
import pandas as pd

import time

from selenium.common.exceptions import TimeoutException
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

import requests

# CAMBIO DE URL

La sección https://tableros.yvera.tur.ar/tablero_ODS/ no se encuentra disponible en la página web. A los fines de hacer el ejercicio decidimos cambiar la url por otra sección de la página: https://tableros.yvera.tur.ar/indice_turistico/

# OBTENCIÓN DE TABLERO DE LOS ÍNDICE TURÍSTICO

Para resolver el ejercicio utilizamos selenium. El objetivo es descargar el archivo csv que está en la página y se condice con la información del tablero. Luego, mostramos el tablero conpandas desde colab. Vamos explicando el código a medida que lo desarrollamos.

In [84]:
#definimos la url
URL = "https://tableros.yvera.tur.ar/indice_turistico/"

# Configurar el navegador de selenium en colab
options = webdriver.ChromeOptions()
options.add_argument('--headless')  # Para ejecutar en segundo plano (sin ventana)
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
options.binary_location = "/usr/bin/chromium-browser"  # Ubicación del navegador Chromium en Colab


# Creamos un objeto driver con las configuraciones anteriores
driver = webdriver.Chrome(options=options)

#Al objeto le pasamos la URL
driver.get(URL)


# En primer lugar debemos encontrar el elemento tablero, dodo que la url no se modifica al ir a esta sección.
# y como es de carga dinámica, nunca encontraríamos el archivo el link al archivo csv.

tablero_element = WebDriverWait(driver, 20).until(
    EC.presence_of_element_located((By.XPATH, "//a[@data-value='TABLERO']"))
)

# Hacemos un scroll hasta el final de la página para que se cargen todos los elementos.
# La carga dinámica mucha veces se produce al scrolear, y, como el link del archivo csv
# Está al final de la página, hacemos un scroll.

driver.execute_script("arguments[0].scrollIntoView();", tablero_element)

# Esperamos antes de hacer click por si hay algun otro elemento temporario que aparece
# sobre TABLERO

time.sleep(2)

#Hacemos click sobre TABLERO
tablero_element.click()

# Damos un tiempo a que se carguen los elementos de TABLERO
time.sleep(2)

# Repetimos el proceso para buscar el boton de download del archivo csv.
# Esta vez lo buscamos por ID.
download_link = WebDriverWait(driver, 20).until(
    EC.element_to_be_clickable((By.ID, "downloadCSV"))
)

# Hacemos otro scroll hasta el final de la página
driver.execute_script("arguments[0].scrollIntoView();", tablero_element)

# Esperamos a que cargen los elementos
time.sleep(2)

# Hacemos click para bajar el archivo
download_link.click()

# Eperamos unos segundos a que descarge el archivo antes de cerrar el driver
time.sleep(5)

# Cerramos el driver
driver.quit()

# MOSTRAMOS EL TABLERO CON PANDAS EN UN DATAFRAME

In [87]:
archivo_csv = 'indice_turistico.csv'

df = pd.read_csv(archivo_csv)

df

,Ranking,Provincia,Departamento/partido,Localidad,Clasificación,ITL
0,1,Ciudad de Buenos Aires,Ciudad de Buenos Aires,Ciudad de Buenos Aires,Consolidado,89.34
1,2,Río Negro,Bariloche,San Carlos de Bariloche,Consolidado,78.67
2,3,Buenos Aires,General Pueyrredón,Mar del Plata,Consolidado,76.96
3,4,Santiago del Estero,Río Hondo,Termas de Río Hondo,Consolidado,73.67
4,5,Santa Cruz,Lago Argentino,El Calafate,Consolidado,73.11
...,...,...,...,...,...,...
1168,1169,Catamarca,Paclín,Las Lajas,Emergente,6.51
1169,1170,Catamarca,Ambato,Colpes,Emergente,6.40
1170,1171,La Rioja,Castro Barros,Aminga,Emergente,5.80
1171,1172,La Rioja,Castro Barros,Los Molinos,Emergente,5.78
